In [18]:
import numpy as np
import pandas as pd
pd.options.display.max_columns = None

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score, f1_score, roc_auc_score, classification_report, confusion_matrix, accuracy_score
from sklearn.feature_selection import SequentialFeatureSelector as SFS, SelectFromModel
from xgboost import XGBClassifier as XGBC
from sklearn.linear_model import RidgeClassifier, RidgeClassifierCV, LogisticRegression
from sklearn.svm import SVC

import torch
import torch.nn as nn
import torch.optim as optim
import tqdm
import copy

In [2]:
# combine=pd.read_csv('Mmerged_icu_mean.csv')
# mergedH=pd.read_csv('MmergedHoldout.csv')

# death0=combine.loc[combine['icu_death']==0]
# death1=combine.loc[combine['icu_death']==1]

# death0['los_bins']=death0.apply(lambda row: 1 if row['los_icu']<=5 else(2 if 5<row['los_icu']<=10 else 3), axis=1 )
# death1['los_bins']=death1.apply(lambda row: 1 if row['los_icu']<=5 else(2 if 5<row['los_icu']<=10 else 3), axis=1 )

# merged, mergedT = train_test_split(death0, 
#                                random_state=1, 
#                                test_size=0.3, 
#                                shuffle=True, 
#                                stratify=death0['los_bins'])

# # merged, mergedT = train_test_split(death1, 
# #                                random_state=1, 
# #                                test_size=0.3, 
# #                                shuffle=True, 
# #                                stratify=death1['los_bins'])

In [3]:
#3class

combine=pd.read_csv('Mmerged_icu_mean.csv')
mergedH=pd.read_csv('MmergedHoldout.csv')

# split to muliticlass
combine['los_bins']=combine.apply(lambda row: 0 if row['los_icu']<=2.5 else(1 if 2.5<row['los_icu']<=5 else 2), axis=1 )
mergedH['los_bins']=mergedH.apply(lambda row: 0 if row['los_icu']<=2.5 else(1 if 2.5<row['los_icu']<=5 else 2), axis=1 )

#train test split
merged, mergedT = train_test_split(combine, 
                               random_state=1, 
                               test_size=0.3, 
                               shuffle=True, 
                               stratify=combine['los_bins'])

In [4]:
print(combine.shape)
print(merged.shape)
print(mergedT.shape)

(18363, 68)
(12854, 68)
(5509, 68)


In [5]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12854 entries, 4725 to 1971
Data columns (total 68 columns):
 #   Column                                                           Non-Null Count  Dtype  
---  ------                                                           --------------  -----  
 0   id                                                               12854 non-null  int64  
 1   hosp_admittime                                                   12854 non-null  object 
 2   hosp_dischtime                                                   12854 non-null  object 
 3   icu_intime                                                       12854 non-null  object 
 4   icu_outtime                                                      12854 non-null  object 
 5   los_icu                                                          12854 non-null  float64
 6   icu_death                                                        12854 non-null  int64  
 7   gender                                

In [6]:
# merged=pd.read_csv('Mmerged.csv')
# mergedT=pd.read_csv('MmergedTest.csv')
# mergedH=pd.read_csv('MmergedHoldout.csv')

In [7]:
cla_merged=merged.drop(['id','hosp_admittime','hosp_dischtime','icu_intime','icu_outtime','icu_death','los_icu','icu_outcome'], axis=1)
cla_mergedT=mergedT.drop(['id','hosp_admittime','hosp_dischtime','icu_intime','icu_outtime','icu_death','los_icu','icu_outcome'],axis=1)
cla_mergedH=mergedH.drop(['id','hosp_admittime','hosp_dischtime','icu_intime','icu_outtime','icu_death','los_icu','icu_outcome'],axis=1)

In [8]:
cla_mergedT.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5509 entries, 2466 to 7346
Data columns (total 60 columns):
 #   Column                                                           Non-Null Count  Dtype  
---  ------                                                           --------------  -----  
 0   gender                                                           5509 non-null   int64  
 1   admission_age                                                    5509 non-null   float64
 2   weight_admit                                                     5509 non-null   float64
 3   height                                                           5509 non-null   float64
 4   charlson_score                                                   5509 non-null   int64  
 5   atrial_fibrillation                                              5509 non-null   int64  
 6   malignant_cancer                                                 5509 non-null   int64  
 7   chf                                    

In [9]:
#split X, y

X_train=cla_merged.drop(['los_bins'], axis=1)
y_train=cla_merged['los_bins']
# y_train=cla_merged.iloc[:,22:22]

X_test=cla_mergedT.drop(['los_bins'], axis=1)
y_test=cla_mergedT['los_bins']
# y_test=cla_mergedT.iloc[:,22:22]

X_hold=cla_mergedH.drop(['los_bins'], axis=1)
y_hold=cla_mergedH['los_bins']

In [10]:
#normalise
num_cols = X_train.columns[X_train.dtypes.apply(lambda c: np.issubdtype(c, np.number))]
scaler = preprocessing.StandardScaler()
X_train[num_cols] = scaler.fit_transform(X_train[num_cols])

num_cols = X_test.columns[X_test.dtypes.apply(lambda c: np.issubdtype(c, np.number))]
scaler = preprocessing.StandardScaler()
X_test[num_cols] = scaler.fit_transform(X_test[num_cols])

num_cols = X_hold.columns[X_hold.dtypes.apply(lambda c: np.issubdtype(c, np.number))]
scaler = preprocessing.StandardScaler()
X_hold[num_cols] = scaler.fit_transform(X_hold[num_cols])

# Ridge

In [20]:
l2=RidgeClassifier()
l2.fit(X_train, y_train)

RidgeClassifier()

In [21]:
sel = SelectFromModel(l2)
sel.fit(X_train, y_train)

selected_feats=sel.get_feature_names_out()
print(selected_feats)
print(len(selected_feats))

['admission_age' 'charlson_score' 'admission_type_URGENT'
 'first_careunit_Medical/Surgical Intensive Care Unit (MICU/SICU)'
 'first_careunit_Neuro Surgical Intensive Care Unit (Neuro SICU)'
 'aniongap' 'bicarbonate' 'bun' 'calcium' 'chloride' 'creatinine' 'sodium'
 'hemoglobin' 'mch' 'mchc' 'mcv' 'rbc' 'wbc' 'pt']
19


In [22]:
Best_l2 = RidgeClassifierCV(alphas=np.logspace(-6, 6, 13))

Best_l2.fit(X_train[selected_feats],y_train)

RidgeClassifierCV(alphas=array([1.e-06, 1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01,
       1.e+02, 1.e+03, 1.e+04, 1.e+05, 1.e+06]))

In [23]:
Best_l2.alpha_

100.0

In [24]:
l2_pred = Best_l2.predict(X_test[selected_feats])
# l2_pp=Best_l2.predict_proba(X_test[selected_feats])

# l2_posProb = l2_pp[:,-1]

# rfc_f1 = f1_score(y_test,rfc_pred)
# rfc_auroc = roc_auc_score(y_test, rfc_pp[:,-1], multi_class="ovr")
# print(rfc_f1)
# print(rfc_auroc)
pd.crosstab(y_test, l2_pred)

col_0,0,1,2
los_bins,,,
0,2181,13,226
1,1372,8,218
2,1168,12,311


In [25]:
print(classification_report(y_test, l2_pred))

              precision    recall  f1-score   support

           0       0.46      0.90      0.61      2420
           1       0.24      0.01      0.01      1598
           2       0.41      0.21      0.28      1491

    accuracy                           0.45      5509
   macro avg       0.37      0.37      0.30      5509
weighted avg       0.38      0.45      0.35      5509



In [26]:
l2_predH = Best_l2.predict(X_hold[selected_feats])
pd.crosstab(y_hold, l2_predH)

col_0,0,1,2
los_bins,,,
0,803,5,81
1,512,2,97
2,425,4,112


In [27]:
print(classification_report(y_hold, l2_predH))

              precision    recall  f1-score   support

           0       0.46      0.90      0.61       889
           1       0.18      0.00      0.01       611
           2       0.39      0.21      0.27       541

    accuracy                           0.45      2041
   macro avg       0.34      0.37      0.30      2041
weighted avg       0.36      0.45      0.34      2041



# select from model

In [28]:
rfc = RandomForestClassifier(random_state=0)
rfc.fit(X_train, y_train)

sel = SelectFromModel(rfc)
sel.fit(X_train, y_train)

SelectFromModel(estimator=RandomForestClassifier(random_state=0))

In [29]:
selected_feats=sel.get_feature_names_out()
print(selected_feats)
print(len(selected_feats))

['admission_age' 'weight_admit' 'height' 'charlson_score' 'aniongap'
 'bicarbonate' 'bun' 'calcium' 'chloride' 'creatinine' 'glucose' 'sodium'
 'potassium' 'hematocrit' 'hemoglobin' 'mch' 'mchc' 'mcv' 'platelet' 'rbc'
 'rdw' 'wbc' 'inr' 'pt' 'ptt']
25


# RF Classifier (select from model)

In [30]:
rfc = RandomForestClassifier(criterion='entropy')

parameters = {'max_depth':np.arange(start=1,stop=10,step=1)}
stratifiedCV = StratifiedKFold(n_splits=5)
Best_rfc = GridSearchCV(
    rfc,
    param_grid=parameters,
    cv=stratifiedCV
)

Best_rfc.fit(X_train[selected_feats],y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=RandomForestClassifier(criterion='entropy'),
             param_grid={'max_depth': array([1, 2, 3, 4, 5, 6, 7, 8, 9])})

In [31]:
Best_rfc.best_estimator_

RandomForestClassifier(criterion='entropy', max_depth=9)

In [32]:
rfc_pred = Best_rfc.predict(X_test[selected_feats])

In [33]:
np.count_nonzero(rfc_pred == 3)

0

In [34]:
#Test-RF with forward selected feature

rfc_pred = Best_rfc.predict(X_test[selected_feats])
# rfc_pp=Best_rfc.predict_proba(X_test[selected_feats])

# rfc_posProb = rfc_pp[:,-1]

# # rfc_f1 = f1_score(y_test,rfc_pred)
# rfc_auroc = roc_auc_score(y_test, rfc_pp[:,-1], multi_class="ovr")
# # print(rfc_f1)
# print(rfc_auroc)
pd.crosstab(y_test, rfc_pred)

col_0,0,1,2
los_bins,,,
0,2074,31,315
1,1192,19,387
2,820,18,653


In [35]:
print(classification_report(y_test, rfc_pred))

              precision    recall  f1-score   support

           0       0.51      0.86      0.64      2420
           1       0.28      0.01      0.02      1598
           2       0.48      0.44      0.46      1491

    accuracy                           0.50      5509
   macro avg       0.42      0.44      0.37      5509
weighted avg       0.43      0.50      0.41      5509



In [36]:
rfc_predH = Best_rfc.predict(X_hold[selected_feats])
pd.crosstab(y_hold, rfc_predH)

col_0,0,1,2
los_bins,,,
0,768,6,115
1,455,10,146
2,301,8,232


In [37]:
print(classification_report(y_hold, rfc_predH))

              precision    recall  f1-score   support

           0       0.50      0.86      0.64       889
           1       0.42      0.02      0.03       611
           2       0.47      0.43      0.45       541

    accuracy                           0.49      2041
   macro avg       0.46      0.44      0.37      2041
weighted avg       0.47      0.49      0.41      2041



# XGB Classifier

In [38]:
xgbc =XGBC()

xgbc.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [39]:
thresholds = np.sort(xgbc.feature_importances_)

In [40]:
thresholds

array([0.00261226, 0.00540418, 0.00900588, 0.00980634, 0.00991814,
       0.01026032, 0.01068008, 0.01124364, 0.01168026, 0.01246598,
       0.01301586, 0.01315564, 0.01321396, 0.01362848, 0.01401611,
       0.01415146, 0.01421236, 0.01430602, 0.01440873, 0.01441966,
       0.01457217, 0.01463204, 0.01473623, 0.01473751, 0.01484406,
       0.01488759, 0.01491375, 0.01499784, 0.01506889, 0.01510221,
       0.01511762, 0.01519116, 0.01519477, 0.01523753, 0.01546703,
       0.01552575, 0.01556992, 0.01584239, 0.01587847, 0.01598733,
       0.0160462 , 0.01608334, 0.01618581, 0.01638884, 0.01658862,
       0.01661846, 0.0171456 , 0.01717224, 0.01775817, 0.01794104,
       0.01800145, 0.0184339 , 0.01869084, 0.02125327, 0.02265598,
       0.03191827, 0.03308707, 0.06061461, 0.07230663], dtype=float32)

In [41]:
for thresh in thresholds:
    selection = SelectFromModel(xgbc, threshold=thresh, prefit=True)
    select_X_train = selection.transform(X_train)
    selection_model = XGBC()
    selection_model.fit(select_X_train, y_train)
    select_X_test = selection.transform(X_test)
    predictions = selection_model.predict(select_X_test)
    accuracy = accuracy_score(y_test, predictions)
    print("Thresh=%.3f, n=%d, Accuracy: %.2f%%" % (thresh, select_X_train.shape[1], accuracy*100.0))

C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


Thresh=0.003, n=59, Accuracy: 48.88%


C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


Thresh=0.005, n=58, Accuracy: 48.68%


C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


...

C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


Thresh=0.033, n=3, Accuracy: 48.74%


C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


Thresh=0.061, n=2, Accuracy: 48.74%
Thresh=0.072, n=1, Accuracy: 47.59%


C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


In [42]:
selected=pd.DataFrame()
selected['col']=X_train.columns
selected['rank']=xgbc.feature_importances_
selected.sort_values(by='rank', ascending=False,inplace=True)

top6=selected[:6]
selected_feats=top6['col'].to_list()
selected_feats

['height',
 'first_careunit_Cardiac Vascular Intensive Care Unit (CVICU)',
 'first_careunit_Neuro Surgical Intensive Care Unit (Neuro SICU)',
 'admission_type_URGENT',
 'first_careunit_Surgical Intensive Care Unit (SICU)',
 'first_careunit_Neuro Intermediate']

In [43]:

xgb = XGBC()

parameters = {
    'n_estimators':np.arange(start=2,stop=20,step=2),
    'max_depth':np.arange(start=2,stop=6,step=1),
    'learning_rate':np.arange(start=0.05,stop=0.4,step=0.05)
}
stratifiedCV = StratifiedKFold(n_splits=5)
Best_xgb = GridSearchCV(
    xgb,
    param_grid=parameters,
    cv=stratifiedCV,
    verbose=1,
    n_jobs=-1
)

Best_xgb.fit(X_train[selected_feats],y_train)

Fitting 5 folds for each of 252 candidates, totalling 1260 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'learning_rate': array([0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35]),
                         'max_depth': array([2, 3, 4, 5]),
                         'n_estimators': array([ 2,  4,  6,  8, 10, 12, 14, 16, 18])},
             verbose=1)

In [44]:
Best_xgb.best_estimator_

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.35000000000000003,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=16, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [45]:

xgb_pred = Best_xgb.predict(X_test[selected_feats])
# xgb_pp=Best_xgb.predict_proba(X_test[selected_feats])

# xgb_posProb = xgb_pp[:,-1]

# xgb_f1 = f1_score(y_test, xgb_pred)
# xgb_auroc = roc_auc_score(y_test, xgb_pp[:,-1])
# print(xgb_f1)
# print(xgb_auroc)
pd.crosstab(y_test, xgb_pred)

col_0,0,1,2
los_bins,,,
0,1755,18,647
1,924,13,661
2,527,11,953


In [46]:
print(classification_report(y_test, xgb_pred))

              precision    recall  f1-score   support

           0       0.55      0.73      0.62      2420
           1       0.31      0.01      0.02      1598
           2       0.42      0.64      0.51      1491

    accuracy                           0.49      5509
   macro avg       0.43      0.46      0.38      5509
weighted avg       0.44      0.49      0.42      5509



In [47]:
xgb_predH = Best_xgb.predict(X_hold[selected_feats])
pd.crosstab(y_hold, xgb_predH)

col_0,0,1,2
los_bins,,,
0,658,8,223
1,370,8,233
2,195,10,336


In [48]:
print(classification_report(y_hold, xgb_predH))

              precision    recall  f1-score   support

           0       0.54      0.74      0.62       889
           1       0.31      0.01      0.03       611
           2       0.42      0.62      0.50       541

    accuracy                           0.49      2041
   macro avg       0.42      0.46      0.38      2041
weighted avg       0.44      0.49      0.41      2041



# Support vector

In [10]:
from sklearn.svm import SVC

names=X_train.columns.to_numpy()
names

array(['gender', 'admission_age', 'weight_admit', 'height',
       'charlson_score', 'atrial_fibrillation', 'malignant_cancer', 'chf',
       'ckd', 'cld', 'copd', 'diabetes', 'hypertension', 'ihd', 'stroke',
       'race_encode_African', 'race_encode_Asian',
       'race_encode_Caucasian', 'race_encode_Hispanic',
       'race_encode_Not Specified', 'race_encode_South American',
       'admission_type_DIRECT EMER.', 'admission_type_DIRECT OBSERVATION',
       'admission_type_ELECTIVE', 'admission_type_EU OBSERVATION',
       'admission_type_EW EMER.', 'admission_type_OBSERVATION ADMIT',
       'admission_type_SURGICAL SAME DAY ADMISSION',
       'admission_type_URGENT',
       'first_careunit_Cardiac Vascular Intensive Care Unit (CVICU)',
       'first_careunit_Coronary Care Unit (CCU)',
       'first_careunit_Medical Intensive Care Unit (MICU)',
       'first_careunit_Medical/Surgical Intensive Care Unit (MICU/SICU)',
       'first_careunit_Neuro Intermediate',
       'first_careunit_

In [11]:
sv = SVC(kernel='linear')
sv.fit(X_train, y_train)

coef =abs(sv.coef_[0])
coef

array([4.36806897e-06, 7.16146465e-06, 2.20318964e-06, 4.70337272e-06,
       8.43044120e-07, 4.77170181e-06, 9.72743089e-06, 5.09053146e-06,
       3.95775018e-06, 1.25195723e-05, 4.81555388e-06, 4.30499910e-06,
       6.29304793e-06, 2.78067827e-06, 5.92813556e-06, 9.06881261e-04,
       4.38413586e-04, 1.25881986e-03, 5.13486684e-04, 9.42147167e-04,
       5.28416890e-02, 3.87489937e-04, 2.24531505e-12, 3.22288350e-04,
       4.98286500e-02, 9.76643389e-04, 6.35601494e-04, 4.93601085e-04,
       7.59843772e-04, 5.30785747e-03, 5.41360632e-03, 7.21685520e-03,
       6.49360914e-03, 1.63537275e-01, 9.17226062e-04, 2.32723146e-03,
       5.90471695e-03, 5.36967800e-03, 2.44899239e-06, 1.60880721e-05,
       4.61154073e-06, 4.83689156e-06, 1.37115949e-06, 6.52088270e-06,
       5.53446975e-06, 3.22435412e-09, 3.53172409e-06, 8.63779732e-06,
       1.49651876e-04, 5.90469841e-04, 3.45439355e-04, 5.59898071e-04,
       1.07171010e-05, 1.27580083e-04, 2.13084332e-05, 1.07950569e-05,
      

In [12]:
sv_rank = pd.DataFrame({'features': names, 'coef': coef})
sv_rank.sort_values(by=['coef'], inplace=True, ascending=False)
sv_rank

,features,coef
33,first_careunit_Neuro Intermediate,1.635373e-01
20,race_encode_South American,5.284169e-02
24,admission_type_EU OBSERVATION,4.982865e-02
31,first_careunit_Medical Intensive Care Unit (MICU),7.216855e-03
32,first_careunit_Medical/Surgical Intensive Care...,6.493609e-03
36,first_careunit_Surgical Intensive Care Unit (S...,5.904717e-03
30,first_careunit_Coronary Care Unit (CCU),5.413606e-03
37,first_careunit_Trauma SICU (TSICU),5.369678e-03
29,first_careunit_Cardiac Vascular Intensive Care...,5.307857e-03
35,first_careunit_Neuro Surgical Intensive Care U...,2.327231e-03


In [13]:
top=sv_rank[:3]
selected_feats=top['features'].to_list()
selected_feats

['first_careunit_Neuro Intermediate',
 'race_encode_South American',
 'admission_type_EU OBSERVATION']

In [14]:
sv = SVC(kernel='rbf')

parameters = {'C':np.arange(0.5,5,step=0.5)}
stratifiedCV = StratifiedKFold(n_splits=5)
sv_f0 = GridSearchCV(
    sv,
    param_grid=parameters,
    scoring='f1',
    cv=stratifiedCV,
    verbose=1,
    n_jobs=-1
)

sv_f0.fit(X_train[selected_feats],y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


C:\Users\kawai\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:952: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan]
  warnings.warn(


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=SVC(), n_jobs=-1,
             param_grid={'C': array([0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5])},
             scoring='f1', verbose=1)

In [15]:
sv_f0.best_estimator_

SVC(C=0.5)

In [16]:
sv_f0.best_score_

nan

In [17]:
sv_f0_pred = sv_f0.predict(X_test[selected_feats])
pd.crosstab(y_test, sv_f0_pred)

col_0,0,1
los_bins,,
0,2406,14
1,1582,16
2,1477,14


In [18]:
print(classification_report(y_test, sv_f0_pred))

              precision    recall  f1-score   support

           0       0.44      0.99      0.61      2420
           1       0.36      0.01      0.02      1598
           2       0.00      0.00      0.00      1491

    accuracy                           0.44      5509
   macro avg       0.27      0.33      0.21      5509
weighted avg       0.30      0.44      0.27      5509



C:\Users\kawai\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kawai\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kawai\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [19]:
sv_f0_predH = sv_f0.predict(X_hold[selected_feats])
pd.crosstab(y_hold, sv_f0_predH)

col_0,0,1
los_bins,,
0,884,5
1,607,4
2,534,7


In [20]:
print(classification_report(y_hold, sv_f0_predH))

              precision    recall  f1-score   support

           0       0.44      0.99      0.61       889
           1       0.25      0.01      0.01       611
           2       0.00      0.00      0.00       541

    accuracy                           0.44      2041
   macro avg       0.23      0.33      0.21      2041
weighted avg       0.26      0.44      0.27      2041



C:\Users\kawai\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kawai\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kawai\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# ensemble

In [26]:
from sklearn.ensemble import StackingClassifier

estimators=[
    ('rf', RandomForestClassifier(max_depth=9, random_state=9)),
    ('xgb', XGBC(n_estimators=16, max_depth=3, learning_rate=0.35)),
    ('sv', SVC(kernel='rbf', C=0.5))
]

ensemb = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(max_iter=1000))

ensemb.fit(X_train, y_train)

StackingClassifier(estimators=[('rf',
                                RandomForestClassifier(max_depth=9,
                                                       random_state=9)),
                               ('xgb',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric=None,
                                              feature_types=None, gamma=None,
                                              gpu_id=None, grow_policy=None,
                                              i...
                                              interaction_constraints=None,
                                              learning_rate=0.35, max_bin=None,
                                              max_cat_threshold=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=None, max_depth=3,
                                              max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              n_estimators=16, n_jobs=None,
                                              num_parallel_tree=None,
                                              predictor=None, random_state=None, ...)),
                               ('sv', SVC(C=0.5))],
                   final_estimator=LogisticRegression(max_iter=1000))

In [27]:
e_pred = emsemb.predict(X_test)
print(classification_report(y_test, e_pred))

              precision    recall  f1-score   support

           0       0.53      0.80      0.64      2420
           1       0.29      0.01      0.01      1598
           2       0.47      0.57      0.51      1491

    accuracy                           0.51      5509
   macro avg       0.43      0.46      0.39      5509
weighted avg       0.44      0.51      0.42      5509



In [28]:
e_pred = emsemb.predict(X_hold)
print(classification_report(y_hold, e_pred))

              precision    recall  f1-score   support

           0       0.54      0.83      0.65       889
           1       0.44      0.01      0.01       611
           2       0.47      0.57      0.52       541

    accuracy                           0.52      2041
   macro avg       0.48      0.47      0.39      2041
weighted avg       0.49      0.52      0.43      2041



In [29]:
estimators=[
    ('rf', RandomForestClassifier(max_depth=9, random_state=9)),
    ('xgb', XGBC(n_estimators=16, max_depth=3, learning_rate=0.35)),
]

emsemb = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(max_iter=1000))

emsemb.fit(X_train, y_train)

StackingClassifier(estimators=[('rf',
                                RandomForestClassifier(max_depth=9,
                                                       random_state=9)),
                               ('xgb',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric=None,
                                              feature_types=None, gamma=None,
                                              gpu_id=None, grow_policy=None,
                                              i...
                                              interaction_constraints=None,
                                              learning_rate=0.35, max_bin=None,
                                              max_cat_threshold=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=None, max_depth=3,
                                              max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              n_estimators=16, n_jobs=None,
                                              num_parallel_tree=None,
                                              predictor=None, random_state=None, ...))],
                   final_estimator=LogisticRegression(max_iter=1000))

In [30]:
e_pred = emsemb.predict(X_hold)
print(classification_report(y_hold, e_pred))

              precision    recall  f1-score   support

           0       0.54      0.83      0.65       889
           1       0.44      0.01      0.01       611
           2       0.47      0.57      0.52       541

    accuracy                           0.52      2041
   macro avg       0.48      0.47      0.39      2041
weighted avg       0.49      0.52      0.43      2041



In [31]:
e_pred = emsemb.predict(X_hold)
print(classification_report(y_hold, e_pred))

              precision    recall  f1-score   support

           0       0.54      0.83      0.65       889
           1       0.44      0.01      0.01       611
           2       0.47      0.57      0.52       541

    accuracy                           0.52      2041
   macro avg       0.48      0.47      0.39      2041
weighted avg       0.49      0.52      0.43      2041



In [32]:
estimators=[
    ('rf', RandomForestClassifier(max_depth=9, random_state=9)),
    ('xgb', XGBC(n_estimators=16, max_depth=3, learning_rate=0.35)),
    ('sv', SVC(kernel='rbf', C=0.5))
]

ensemb = StackingClassifier(estimators=estimators, final_estimator=RandomForestClassifier())

ensemb.fit(X_train, y_train)

StackingClassifier(estimators=[('rf',
                                RandomForestClassifier(max_depth=9,
                                                       random_state=9)),
                               ('xgb',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric=None,
                                              feature_types=None, gamma=None,
                                              gpu_id=None, grow_policy=None,
                                              i...
                                              interaction_constraints=None,
                                              learning_rate=0.35, max_bin=None,
                                              max_cat_threshold=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=None, max_depth=3,
                                              max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              n_estimators=16, n_jobs=None,
                                              num_parallel_tree=None,
                                              predictor=None, random_state=None, ...)),
                               ('sv', SVC(C=0.5))],
                   final_estimator=RandomForestClassifier())

In [33]:
e_pred = emsemb.predict(X_test)
print(classification_report(y_test, e_pred))

              precision    recall  f1-score   support

           0       0.53      0.80      0.64      2420
           1       0.29      0.01      0.01      1598
           2       0.47      0.57      0.51      1491

    accuracy                           0.51      5509
   macro avg       0.43      0.46      0.39      5509
weighted avg       0.44      0.51      0.42      5509



In [34]:
e_pred = emsemb.predict(X_hold)
print(classification_report(y_hold, e_pred))

              precision    recall  f1-score   support

           0       0.54      0.83      0.65       889
           1       0.44      0.01      0.01       611
           2       0.47      0.57      0.52       541

    accuracy                           0.52      2041
   macro avg       0.48      0.47      0.39      2041
weighted avg       0.49      0.52      0.43      2041

